In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data_input = np.arange(10000)
training_set = pd.DataFrame(data_input)


In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
X_train = []
y_train = []
for i in range(60, 10000):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [6]:
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(LSTM(units = 50))
model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train, y_train, epochs = 10, batch_size = 32)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - loss: 0.0234
Epoch 2/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 1.4856e-05
Epoch 3/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - loss: 1.0938e-05
Epoch 4/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 22s 53ms/step - loss: 7.6537e-06
Epoch 5/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 5.1449e-06
Epoch 6/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - loss: 2.6021e-06
Epoch 7/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - loss: 4.3564e-07
Epoch 8/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 1.0619e-07
Epoch 9/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - loss: 1.3920e-07
Epoch 10/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - loss: 1.7214e-07


In [7]:
y_predict = model.predict(X_train)

311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step


In [9]:
pd.DataFrame(sc.inverse_transform(y_predict))

,0
0,68.013725
1,68.997940
2,69.982269
3,70.966507
4,71.950783
...,...
9935,10002.216797
9936,10003.227539
9937,10004.237305
9938,10005.248047


## Code Summary and Explanation

This notebook demonstrates a simple example of using a Long Short-Term Memory (LSTM) neural network to predict a sequence.

1.  **Import Libraries:** Necessary libraries like `numpy`, `matplotlib.pyplot`, `pandas`, `MinMaxScaler`, `Sequential`, `Dense`, `LSTM`, and `Dropout` are imported.
2.  **Prepare Data:** A sequence of numbers from 0 to 9999 is generated using `numpy.arange` and converted into a pandas DataFrame.
3.  **Scale Data:** The data is scaled using `MinMaxScaler` to a range between 0 and 1. This is a common practice in neural networks.
4.  **Create Training Data:** The scaled data is prepared for the LSTM model by creating sequences of 60 previous data points (`X_train`) to predict the next data point (`y_train`). The data is then reshaped into a 3D format required by the LSTM layer.
5.  **Build LSTM Model:** A sequential LSTM model is built with two LSTM layers and one dense output layer.
    *   The first LSTM layer has `return_sequences=True` to output a sequence for the next LSTM layer.
    *   The second LSTM layer outputs a single value.
    *   The Dense layer provides the final prediction.
6.  **Compile and Train Model:** The model is compiled using the 'adam' optimizer and 'mean_squared_error' loss function. It is then trained on the prepared training data for 10 epochs with a batch size of 32.
7.  **Make Predictions:** The trained model is used to make predictions on the training data (`X_train`).
8.  **Inverse Transform Predictions:** The predictions, which are in the scaled range, are inverse transformed back to the original scale using the `inverse_transform` method of the `MinMaxScaler`.

In essence, the code builds and trains an LSTM model to learn the pattern in the generated sequence and then uses the trained model to predict the next values in the sequence.

Now that we have a trained model, we can use it to make predictions on new data. We need to prepare the new input data in the same way as the training data (scaling and reshaping).

In [10]:
# Let's create a sample new input sequence of 60 values from the original data range
# In a real-world scenario, this would be new data you want to predict on
new_input_data = data_input[9940:10000] # Taking the last 60 values from the original data as an example

# Reshape the input data to be a DataFrame for scaling
new_input_data = pd.DataFrame(new_input_data)

# Scale the new input data using the same scaler fitted on the training data
new_input_scaled = sc.transform(new_input_data)

# Reshape the scaled input data to the 3D format expected by the LSTM model (number of sequences, time steps, features)
new_input_reshaped = np.reshape(new_input_scaled, (1, new_input_scaled.shape[0], 1))

# Make a prediction with the trained model
predicted_value_scaled = model.predict(new_input_reshaped)

# Inverse transform the predicted value back to the original scale
predicted_value = sc.inverse_transform(predicted_value_scaled)

print(f"The predicted value is: {predicted_value[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
The predicted value is: 10007.2685546875


Let's consider a smaller example. If we were using a look-back of 3 instead of 60:

*   **Input sequence:** `[10, 11, 12]`
*   **Model prediction:** The model would predict the next value in the sequence, which should be close to `13`.

In our current notebook, with a look-back of 60, the model is given the last 60 values from our generated data (9940 to 9999). Based on this input sequence:

*   **Input sequence:** `[9940, 9941, 9942, ..., 9999]` (a sequence of 60 numbers)
*   **Model prediction:** The model predicts the next value, which the output showed was approximately `10007.26`. This is close to the expected next value of `10000` if the sequence continued linearly, but the model's prediction is influenced by the training data and the nature of the LSTM.